This notebook shows how to use the one and two body integrals computed using OpenFermion and a quantum chemistry package like pySCF in order to construct a fermionic Hamiltonian on the QLM.

OpenFermion and qat-dqs use slightly different conventions for the definition of the second quantized Hamiltonian:

$$ H_\mathrm{OpenFermion}=\sum_{pq}h_{pq}c_p^\dagger c_q + \sum_{pqrs} \tilde{h}_{pqrs}c_p^\dagger c_q^\dagger c_r c_s $$

$$ H_\mathrm{QLM}=\sum_{pq}h_{pq}c_p^\dagger c_q + \frac{1}{2} \sum_{pqrs} h_{pqrs}c_p^\dagger c_q^\dagger c_r c_s $$

In the notebook below, we take the example of a $H_2$ molecule.

In [ ]:
from openfermion.hamiltonians import MolecularData
from openfermionpyscf import run_pyscf

diatomic_bond_length = 0.75
geometry = [('H', (0., 0., 0.)), ('H', (0., 0., diatomic_bond_length))]

molecule = MolecularData(geometry, basis='sto-3g', multiplicity=1)
_ = run_pyscf(molecule, run_fci=True)
openfermion_hamiltonian = molecule.get_molecular_hamiltonian()

from qat.dqs.hamiltonians import FermionicHamiltonian
qlm_hamiltonian = FermionicHamiltonian(hpq=openfermion_hamiltonian.one_body_tensor,
                                       hpqrs=2 * openfermion_hamiltonian.two_body_tensor)

We can check that both Hamiltonians have the same ground-state energy:

In [ ]:
import numpy as np
from openfermion.transforms import get_sparse_operator

matrix = get_sparse_operator(openfermion_hamiltonian)
eigvals = np.linalg.eigvals(matrix.A)
print("OpenFermion GS energy:", min(eigvals))

eigvals, _ = qlm_hamiltonian.get_eigenenergy()
print("QLM GS energy:", min(eigvals) + openfermion_hamiltonian.constant)